# Laborator I

## Instalare sly

Pentru a instala python-sly (care este o reimplementare a programelor lex si yacc) trebuie sa aveti instalat python3 si apoi trebuie sa instalati pachetul sly: 

```$ sudo apt-get update && sudo apt-get install python3.7 python3-pip && sudo pip install sly```

## Verificare instalare

Pentru a verifica instalarea rulati urmatorul cod:

In [1]:
import sys
from sly import Lexer, Parser

print("Python version:", sys.version)

Python version: 3.7.4 (default, Aug 13 2019, 20:35:49) 
[GCC 7.3.0]


## Constructia unui lexer simplu

Pentru a construi un lexer simplu trebuie sa definim o clasa si sa derivam din clasa Lexer. Dupa aceasta trebuie sa definim tipurile de tokeni folositi si sa definim o expresie regulata (testare regex: https://pythex.org/) pentru fiecare dintre acestia. Apoi trebuie sa rulam lexer-ul pentru a primi fiecare token dintr-un input.

Ca o prima problema, sa spunem ca vrem sa citim fie numere fie cuvinte si atunci cand am ajuns la un spatiu sa spunem daca ceea ce am citit este un cuvant sau un numar.

In [2]:
class WordOrNumberLexer(Lexer):
    tokens = { WORD, NUMBER }   # avem doar doua tipuri de tokeni
    ignore = ' \t'              # nu ne intereseaza spatiile sau tab-ul

    # Expresii regulate pentru fiecare tip de token
    WORD = r'[a-zA-Z_][a-zA-Z0-9_]*'
    NUMBER = r'\d+'

    # Puteti sa ignorati si pattern-uri, nu doar litere simple cum avem mai sus
    ignore_newline = r'\n+'

    # Puteti sa numarati cate newline-uri aveti in fisierul text
    def ignore_newline(self, t):
        self.lineno += t.value.count('\n')

    # Daca apare altceva inafara de expresiile regulate definite mai sus
    # afisam o eroare intrucat nu putem parsa mai departe.
    # Programul de mai jos doar trece peste aceasta eroare.
    def error(self, t):
        print("Illegal character '%s'" % t.value[0])
        self.index += 1

Avand clasa definita, urmeaza sa implementam functia main care citeste de la tastatura input-uri si afiseaza pentru fiecare dintre acestea in ce categorie e.

In [3]:
lexer = WordOrNumberLexer()
while True:
    try:
        text = input('input > ')
    except KeyboardInterrupt:
        break
    if text:
        for token in lexer.tokenize(text):
            print(token)
        print()

input > test
Token(type='WORD', value='test', lineno=1, index=0)

input > 123
Token(type='NUMBER', value='123', lineno=1, index=0)

input > test 123 ana are 4 mere
Token(type='WORD', value='test', lineno=1, index=0)
Token(type='NUMBER', value='123', lineno=1, index=5)
Token(type='WORD', value='ana', lineno=1, index=9)
Token(type='WORD', value='are', lineno=1, index=13)
Token(type='NUMBER', value='4', lineno=1, index=17)
Token(type='WORD', value='mere', lineno=1, index=19)



Sa modificam clasa pentru a numara cate cuvinte si cate numere avem.
De asemenea nu mai citim de la tastatura ci vom presupune ca am citit din fisier.

In [4]:
class WordOrNumberLexer(Lexer):
    tokens = { WORD, NUMBER }   # avem doar doua tipuri de tokeni
    ignore = ' \t'              # nu ne intereseaza spatiile sau tab-ul

    def __init__(self):
        self.words_count = 0
        self.numbers_count = 0
        
        self.words = []
        self.numbers = []
    
    # Expresii regulate pentru fiecare tip de token
    WORD = r'[a-zA-Z_][a-zA-Z0-9_]*'
    NUMBER = r'\d+'
    
    def WORD(self, w):
        self.words_count += 1
        self.words.append(w.value)
        
        return w
        
    def NUMBER(self, n):
        self.numbers_count += 1
        self.numbers.append(n.value)
        
        return n
    
    # Puteti sa ignorati si pattern-uri, nu doar litere simple cum avem mai sus
    ignore_newline = r'\n+'

    # Puteti sa numarati cate newline-uri aveti in fisierul text
    def ignore_newline(self, t):
        self.lineno += t.value.count('\n')

    # Daca apare altceva inafara de expresiile regulate definite mai sus
    # afisam o eroare intrucat nu putem parsa mai departe.
    # Programul de mai jos doar trece pe aceasta eroare.
    def error(self, t):
        print("Illegal character '%s'" % t.value[0])
        self.index += 1

In [5]:
input_text = '1337 test other 1122 1 3 here'
lexer = WordOrNumberLexer()

for token in lexer.tokenize(input_text):
    print(token)

print()    
print('Numbers_count = ', lexer.numbers_count, 'Numbers = ', lexer.numbers)
print('Words_count = ', lexer.words_count, 'Words = ', lexer.words)

Token(type='NUMBER', value='1337', lineno=1, index=0)
Token(type='WORD', value='test', lineno=1, index=5)
Token(type='WORD', value='other', lineno=1, index=10)
Token(type='NUMBER', value='1122', lineno=1, index=16)
Token(type='NUMBER', value='1', lineno=1, index=21)
Token(type='NUMBER', value='3', lineno=1, index=23)
Token(type='WORD', value='here', lineno=1, index=25)

Numbers_count =  4 Numbers =  ['1337', '1122', '1', '3']
Words_count =  3 Words =  ['test', 'other', 'here']


Dupa cum puteti vedea numerele salvate in lista numbers sunt inca string-uri. Daca vreti sa obtineti suma lor trebuie mai intai sa le convertiti la numere intregi (folosind functia int('123')) si apoi de fiecare data cand cititi un numar sa il adaugati la suma. Alternativ, puteti face asta si in programul principal.

### Exercitiu

Primind un for-uri si if-uri afisati tokenii lor. Pentru a usura implementarea la acest moment, vom considera ca avem doua cuvinte cheie: for si if; instructiunile sunt date ca a=b si conditiile a<b; si aveti paranteze si acolade deschise si inchise. De exemplu daca primiti for(i=0; i<9; i=i+1) { if(i>2) a=a+i; if (i<3) a=a+2; } trebuie sa afisati FOR PDESCHISA INSTRUCTIUNE CONDITIE INSTRUCTIUNE PINCHISA ADESCHISA IF PDESCHISA CONDITIE PINCHISA ...

In [6]:
class ForIfLexer(Lexer):
    tokens = { FOR_TOKEN, IF_TOKEN, INSTRUCTION, CONDITION, OPENP, CLOSEDP, OPENCB, CLOSEDCB }
    ignore = ' \t'
    
    # Expresii regulate pentru fiecare tip de token
    FOR_TOKEN = r'for'
    IF_TOKEN = r'if'
    INSTRUCTION = r'[a-zA-Z_][a-zA-Z0-9_]*\=([a-zA-Z_][a-zA-Z0-9_]*|\d+)(\+([a-zA-Z_][a-zA-Z0-9_]*|\d+))?;?'
    CONDITION = r'[a-zA-Z_][a-zA-Z0-9_]*(<|>|<\=|>\=)([a-zA-Z_][a-zA-Z0-9_]*|\d+);?'
    OPENP = r'\('
    CLOSEDP = r'\)'
    OPENCB = r'{'
    CLOSEDCB = r'}'
    
    # Puteti sa ignorati si pattern-uri, nu doar litere simple cum avem mai sus
    ignore_newline = r'\n+'

    # Puteti sa numarati cate newline-uri aveti in fisierul text
    def ignore_newline(self, t):
        self.lineno += t.value.count('\n')

    # Daca apare altceva inafara de expresiile regulate definite mai sus
    # afisam o eroare intrucat nu putem parsa mai departe.
    # Programul de mai jos doar trece pe aceasta eroare.
    def error(self, t):
        print("Illegal character '%s'" % t.value[0])
        self.index += 1

In [7]:
input_text = 'for(i=0; i<9; i=i+1) { if(i>2) a=a+i; if (i<3) a=a+2; } '
lexer = ForIfLexer()

tokens_value = []
for token in lexer.tokenize(input_text):
    print(token)
    tokens_value.append(token.type)

print()
print(' '.join(tokens_value))

Token(type='FOR_TOKEN', value='for', lineno=1, index=0)
Token(type='OPENP', value='(', lineno=1, index=3)
Token(type='INSTRUCTION', value='i=0;', lineno=1, index=4)
Token(type='CONDITION', value='i<9;', lineno=1, index=9)
Token(type='INSTRUCTION', value='i=i+1', lineno=1, index=14)
Token(type='CLOSEDP', value=')', lineno=1, index=19)
Token(type='OPENCB', value='{', lineno=1, index=21)
Token(type='IF_TOKEN', value='if', lineno=1, index=23)
Token(type='OPENP', value='(', lineno=1, index=25)
Token(type='CONDITION', value='i>2', lineno=1, index=26)
Token(type='CLOSEDP', value=')', lineno=1, index=29)
Token(type='INSTRUCTION', value='a=a+i;', lineno=1, index=31)
Token(type='IF_TOKEN', value='if', lineno=1, index=38)
Token(type='OPENP', value='(', lineno=1, index=41)
Token(type='CONDITION', value='i<3', lineno=1, index=42)
Token(type='CLOSEDP', value=')', lineno=1, index=45)
Token(type='INSTRUCTION', value='a=a+2;', lineno=1, index=47)
Token(type='CLOSEDCB', value='}', lineno=1, index=54)

F

### Exercitiu

Construiti un lexer care primeste numere in baza b (2 <= b <= 10) si **converteste fiecare numar in baza 10**.

Numerele primite sunt de forma baza\\$numar. Daca baza = 10, 10\\$ poate lipsi si poate ramana doar numarul.

Exemple de numere: 2\\$1101 4\\$1132 006\\$5155 10\\$1918. Ultimul numar poate fi dat si ca 1918.


In [8]:
class ConversionLexer(Lexer):
    tokens = {}
    ignore = ' \t'
    
    def error(self, t):
        print("Illegal character '%s'" % t.value[0])
        self.index += 1

In [9]:
input_text = '2$1101 4$1132 006$5155 10$1918'
lexer = ConversionLexer()

for token in lexer.tokenize(input_text):
    print(token)

AttributeError: type object 'ConversionLexer' has no attribute '_master_re'

### Exercitiu

Notatia condensata poate fi folosita pentru a utiliza mai putina memorie atunci cand dorim sa salvam numere. Principiul condensarii se bazeaza pe faptul ca putem reprezenta un numar relativ la o baza. De exemplu daca avem urmatoarea secventa de numere si baza = 150:

> S = 149 154 154 154 147 147 145 152 153 150 150 148 162

Putem condensa numerele in functie de baza:

> S = 150-1 150+4 150+4 150+4 150-3 150-3 150-5 150+2 150+3 150 150 150-2 150+12

Atunci cand sunt mai multe aparitii ale aceluiasi numar punem + (respectiv -) de cate ori apare numarul respectiv.
Daca numarul este chiar baza punem 0.

> CS = 150\\$-1b+++4b--3b-5b+2b+3b00b-2b+12b

Unde b reprezinta un spatiu.

Voi trebuie sa scrieti un program care va converteste o expresie in din forma CS in S.

In [ ]:
class CondensedNotationLexer(Lexer):
    tokens = {}
    ignore = ' \t'
    
    def error(self, t):
        print("Illegal character '%s'" % t.value[0])
        self.index += 1

In [ ]:
input_text = '150$-1 +++4 --3 -5 +2 +3 00 -2 +12 '
lexer = CondensedNotationLexer()

for token in lexer.tokenize(input_text):
    print(token)